In [6]:
import numpy as np
import sys
import itertools as itertools
import time
sys.path.append("..")
sys.path.append("../..")

import matplotlib.pyplot as plt
from python.Heisenberg import *
from python.DMRG import DMRG
from python.Zippers import MPO_to_Hamiltonian, contract_MPS
from python.Canonical_Form import get_Neumann_entropy
from python.Backend import Backend

In [7]:
n_sites = 50

In [8]:
device = "numpy"
# device = "torch"

bk = Backend(device)

NKeep = 50
NSweep = 20
Krylov_bases = 5 #* 5 is usually enough
Lanczos_cutoff = 1e-2
iterative_diag = True
two_site = True
verbose = True

In [9]:
import numpy.typing as npt
import pickle
from dataclasses import dataclass, field, asdict, replace
from copy import deepcopy
from typing import Any, Optional
from pathlib import Path


@dataclass
class Params:
    L: int
    J1: float
    J2: float
    NKeep: int
    NSweep: int
    diag: bool
    Krylov: int
    cutoff: float
    two: bool
    
    def to_log(self) -> str:
        log_dict = asdict(self)
        
        log_dict['J1'] = round(self.J1, 2)
        log_dict['J2'] = round(self.J2, 2)
        log_dict['cutoff'] = round(self.cutoff, 3)
        
        # Filter out keys where the value is None
        filtered_log_dict = {key: log for key, log in log_dict.items() if log is not None}
        
        # Create log string
        return "_".join(
            f"{key}={log}" for key, log in filtered_log_dict.items()
        )


@dataclass
class Results:
    ground_MPS: list[npt.NDArray]
    ground_state_energy: float
    Neumann_entropy: float
    ground_energies: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    spin_z: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    spin_correlations: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    
    
    def to_array(self):
        return replace(
            self, **{field: np.array(getattr(self, field))
            if isinstance(getattr(self, field), list) else getattr(self, field)
            for field in asdict(self)}
        )
        
    def self_destruct(self):
        for field_name in self.__dataclass_fields__:
            field_value = getattr(self, field_name)
            if isinstance(field_value, list):
                field_value.clear()  # Clear list contents
            else:
                setattr(self, field_name, None)  # Set to None if not a list



In [10]:
J1_start = -4
J1_end = 4
delta_J1 = 0.1

J1s = bk.arange(J1_start, J1_end+delta_J1, delta_J1)

J2_start = -4
J2_end = 4
delta_J2 = 0.1

J2s = bk.arange(J2_start, J2_end+delta_J2, delta_J2)


In [11]:
for it1, J1 in enumerate(J1s):
    for it2, J2 in enumerate(J2s):
        
        if verbose:
            print(f"J1={round(J1,2)} | J2={round(J2,2)}")
        
        Hamiltonian = nn_Heisenberg_model(
            bk=bk, n_sites=n_sites,
            J1=J1, J2=J2,
        )
        
        ground_energies, ground_times, ground_MPS = DMRG(
            bk = bk,
            Hamiltonian = Hamiltonian,
            NKeep = NKeep,
            NSweep = NSweep,
            Krylov_bases = Krylov_bases,
            Lanczos_cutoff = Lanczos_cutoff,
            iterative_diag = True,
            two_site = two_site,
            verbose = verbose,
            tol = 1e-8,
        )
        
        param = Params(
            L = n_sites,
            J1 = J1,
            J2 = J2,
            NKeep = NKeep,
            NSweep = NSweep,
            diag = iterative_diag,
            Krylov = Krylov_bases,
            cutoff = Lanczos_cutoff,
            two = two_site
        )
        
        # dir_name = Path(f"data")
        dir_name = Path(f"{device}/data")
        dir_name.mkdir(parents=True, exist_ok=True)
        
        filename = f"{param.to_log()}"
        ground_state_energy = ground_energies[-1]
        Neumann_entropy = get_Neumann_entropy(ground_MPS, bk=bk)
        spin_z = get_spin_z(ground_MPS, bk)
        
        if verbose:
            print(f"computing spin correlations: ", end="")
            now = time.perf_counter()
        spin_correlations = get_spin_correlations(ground_MPS, bk)
        if verbose:
            print(f"{round(time.perf_counter()-now,3)}s")
        
        result = Results(
            ground_MPS = [],
            ground_state_energy = ground_state_energy,
            ground_energies = ground_energies,
            Neumann_entropy = Neumann_entropy,
            spin_z = spin_z,
            spin_correlations = spin_correlations,
        )
        
        output: dict[str, Any] = {}
        output.update(asdict(param))
        output.update(asdict(result))
        
        with open(dir_name / f"{filename}.pkl", "wb") as file:
            pickle.dump(output, file)
        

J1=-4.0 | J2=-4.0
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-97.0 | time=0js
iter=1 | energy=(-97+0j) | time=0.17848289402900264s
computing spin correlations: 2.139s
J1=-4.0 | J2=-3.9
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-95.8 | time=0js
iter=1 | energy=(-95.79999999999994+0j) | time=0.1293949949904345s
computing spin correlations: 2.129s
J1=-4.0 | J2=-3.8
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-94.6 | time=0js
iter=1 | energy=(-94.60000000000008+0j) | time=0.12823439500061795s
computing spin correlations: 2.117s
J1=-4.0 | J2=-3.7
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-93.4 | time=0js
iter=1 | energy=(-93.39999999999992+0j) | time=0.1477043